In [1]:
cd /kaggle/input/dataset-eng-por

/kaggle/input/dataset-eng-por


In [2]:
import pdb
import torch
import itertools
import numpy as np
import torch.nn as nn
from collections import Counter
from utils_PT import (sentences, train_dataset, val_dataset, train_loader, val_loader,
                   tokenizer_eng, tokenizer_por, masked_loss, masked_acc, ids_to_text, encode_sample, pt_lower_and_split_punct)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data preparation

In [4]:
english_sentences, portuguese_sentences = sentences

print(f"English (to translate) sentence:\n\n{english_sentences[-5]}\n")
print(f"Portuguese (translation) sentence:\n\n{portuguese_sentences[-5]}")

English (to translate) sentence:

No matter how much you try to convince people that chocolate is vanilla, it'll still be chocolate, even though you may manage to convince yourself and a few others that it's vanilla.

Portuguese (translation) sentence:

Não importa o quanto você tenta convencer os outros de que chocolate é baunilha, ele ainda será chocolate, mesmo que você possa convencer a si mesmo e poucos outros de que é baunilha.


In [5]:
source = pt_lower_and_split_punct(english_sentences[-5])
print(f"source: {source}\n")

encoded = tokenizer_eng.encode(source[0]).ids
print(f"encoded: {encoded}\n")
print(f"len(encoded): {len(encoded)}")

decoded = tokenizer_eng.decode(encoded)
print(f"decoded: {decoded}\n")
print(f"len(decoded): {len(decoded)}")

source: ['[SOS] no matter how much you try to convince people that chocolate is vanilla ,  itll still be chocolate ,  even though you may manage to convince yourself and a few others that its vanilla . [EOS]']

encoded: [3, 89, 473, 49, 106, 8, 220, 7, 1060, 135, 13, 1107, 12, 4151, 24, 502, 103, 33, 1107, 24, 251, 1169, 8, 219, 2558, 7, 1060, 344, 43, 11, 365, 791, 13, 52, 4151, 4, 2]

len(encoded): 37
decoded: [SOS] no matter how much you try to convince people that chocolate is vanilla , itll still be chocolate , even though you may manage to convince yourself and a few others that its vanilla . [EOS]

len(decoded): 195


In [6]:
del portuguese_sentences
del english_sentences
del sentences

In [7]:
ten_words_eng_vocab = sorted(tokenizer_eng.get_vocab().items(), key=lambda item: item[1])[:10]
ten_words_por_vocab = sorted(tokenizer_por.get_vocab().items(), key=lambda item: item[1])[:10]
print(f"First 10 words of the english vocabulary:\n\n{[token[0] for token in ten_words_eng_vocab]}\n")
print(f"First 10 words of the portuguese vocabulary:\n\n{[token[0] for token in ten_words_por_vocab]}")

First 10 words of the english vocabulary:

['[PAD]', '[UNK]', '[EOS]', '[SOS]', '.', 'tom', 'i', 'to', 'you', 'the']

First 10 words of the portuguese vocabulary:

['[PAD]', '[UNK]', '[EOS]', '[SOS]', '.', 'tom', 'que', 'o', 'nao', 'eu']


In [8]:
# Size of the vocabulary
vocab_size_por = tokenizer_eng.get_vocab_size()
vocab_size_eng = tokenizer_eng.get_vocab_size()

print(f"Portuguese vocabulary is made up of {vocab_size_por} words")
print(f"English vocabulary is made up of {vocab_size_eng} words")

Portuguese vocabulary is made up of 12000 words
English vocabulary is made up of 12000 words


In [9]:
def word_to_id(token):
    return tokenizer_por.token_to_id(token)


def ids_to_words(id):
    return tokenizer_por.id_to_token(id)

In [10]:
unk_id = word_to_id("[UNK]")
sos_id = word_to_id("[SOS]")
eos_id = word_to_id("[EOS]")
baunilha_id = word_to_id("baunilha")

print(f"The id for the [UNK] token is {unk_id}")
print(f"The id for the [SOS] token is {sos_id}")
print(f"The id for the [EOS] token is {eos_id}")
print(f"The id for baunilha (vanilla) is {baunilha_id}")

The id for the [UNK] token is 1
The id for the [SOS] token is 3
The id for the [EOS] token is 2
The id for baunilha (vanilla) is 5242


In [11]:
(to_translate, sr_translation), translation = next(iter(train_loader))

print(f"Tokenized english sentence:\n{to_translate[0, :].numpy()}\n\n")
print(f"Tokenized portuguese sentence (shifted to the right):\n{sr_translation[0, :].numpy()}\n\n")
print(f"Tokenized portuguese sentence:\n{translation[0, :].numpy()}\n\n")

print(tokenizer_eng.decode(to_translate[0, :].numpy()))
print(tokenizer_por.decode(sr_translation[0, :].numpy()))
print(tokenizer_por.decode(translation[0, :].numpy()))

Tokenized english sentence:
[   3  173   46   66  282   66   22 2167  793    4    2    0    0    0
    0    0    0    0    0]


Tokenized portuguese sentence (shifted to the right):
[  3 103 171   6  12 744 378   4   0   0   0   0   0   0   0   0   0   0
   0]


Tokenized portuguese sentence:
[103 171   6  12 744 378   4   2   0   0   0   0   0   0   0   0   0   0
   0]


[SOS] lets go as soon as it stops raining . [EOS]
[SOS] vamos assim que a chuva parar .
vamos assim que a chuva parar . [EOS]


# Encoder

In [12]:
VOCAB_SIZE = 12000
UNITS = 256

In [13]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, units):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, units, padding_idx=0)
        self.rnn = nn.LSTM(units, units, bidirectional=True, batch_first=True)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.rnn(x)
        # Summarizing the bidirectional RNNs to follow the TF version
        forward_output = x[:, :, :UNITS]
        backward_output = x[:, :, UNITS:]
        x = forward_output + backward_output

        return x

In [14]:
encoder = Encoder(VOCAB_SIZE, UNITS)

encoder_output = encoder(to_translate)

print(f'Tensor of sentences in english has shape: {to_translate.shape}\n')
print(f'Encoder output has shape: {encoder_output.shape}')

Tensor of sentences in english has shape: torch.Size([64, 19])

Encoder output has shape: torch.Size([64, 19, 256])


# Cross Attention

In [15]:
class CrossAttention(nn.Module):
    def __init__(self, units):
        super().__init__()

        self.mha = nn.MultiheadAttention(units, 1, batch_first=True)
        self.layernorm = nn.LayerNorm(units)

    def forward(self, context, target):
        attn_output = self.mha(query=target,key=context, value=context)
        x = target + attn_output[0] # [0] because we only need the attention output and no weights
        x = self.layernorm(x) 

        return x

In [16]:
attention_layer = CrossAttention(UNITS)

sr_translation_embed = nn.Embedding(VOCAB_SIZE, UNITS, 0)(sr_translation)

attention_result = attention_layer(encoder_output, sr_translation_embed)

print(f'Tensor of contexts has shape: {encoder_output.shape}')
print(f'Tensor of translations has shape: {sr_translation_embed.shape}')
print(f'Tensor of attention scores has shape: {attention_result.shape}')

Tensor of contexts has shape: torch.Size([64, 19, 256])
Tensor of translations has shape: torch.Size([64, 19, 256])
Tensor of attention scores has shape: torch.Size([64, 19, 256])


# Decoder

In [17]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, units):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, units, padding_idx=0)
        self.pre_attention_rnn = nn.LSTM(units, units, batch_first=True)
        self.attention = CrossAttention(units)
        self.post_attention_rnn = nn.LSTM(units, units, batch_first=True)
        self.output_layer = nn.Linear(units, vocab_size)
        self.activation = nn.LogSoftmax(dim=-1)

    def forward(self, context, target_in, state=None, return_state=False):
        x = self.embedding(target_in)
        x, (hidden_state, cell_state) = self.pre_attention_rnn(x, state)
        x = self.attention(context, x)
        x, _ = self.post_attention_rnn(x)
        x = self.output_layer(x)
        logits = self.activation(x)

        if return_state:
            return logits, [hidden_state, cell_state]

        return logits

In [18]:
decoder = Decoder(VOCAB_SIZE, UNITS)

logits = decoder(encoder_output, sr_translation)

print(f'Tensor of contexts has shape: {encoder_output.shape}')
print(f'Tensor of right-shifted translations has shape: {sr_translation.shape}')
print(f'Tensor of logits has shape: {logits.shape}')

Tensor of contexts has shape: torch.Size([64, 19, 256])
Tensor of right-shifted translations has shape: torch.Size([64, 19])
Tensor of logits has shape: torch.Size([64, 19, 12000])


# Translator

In [19]:
class Translator(nn.Module):
    def __init__(self, vocab_size, units):
        super().__init__()

        self.encoder = Encoder(vocab_size, units)
        self.decoder = Decoder(vocab_size, units)

    def forward(self, inputs):
        context, targets = inputs

        encoded_context = self.encoder(context)
        logits = self.decoder(encoded_context, targets)

        return logits

In [20]:
translator = Translator(VOCAB_SIZE, UNITS).to(device)

# Loading the model
#translator.load_state_dict(torch.load('/kaggle/working/model_weights.pth', map_location=torch.device(device), weights_only=True))

logits = translator((to_translate.to(device), sr_translation.to(device)))

print(f'Tensor of sentences to translate has shape: {to_translate.shape}')
print(f'Tensor of right-shifted translations has shape: {sr_translation.shape}')
print(f'Tensor of logits has shape: {logits.shape}')

Tensor of sentences to translate has shape: torch.Size([64, 19])
Tensor of right-shifted translations has shape: torch.Size([64, 19])
Tensor of logits has shape: torch.Size([64, 19, 12000])


In [21]:
optimizer = torch.optim.Adam(params=translator.parameters())
criterion = masked_loss
acc = masked_acc

# Training

In [22]:
NUM_EPOCHS = 20
STEPS_PER_EPOCH = 500
VALIDATION_STEPS = 50
patience = 3
best_loss = float('inf')
num_batches_train = len(train_loader)
num_batches_val = len(val_loader)


for epoch in range(NUM_EPOCHS):
    translator.train()
    
    # Mini batch loss
    running_loss_train = 0.0
    running_accuracy_train = 0.0
    running_loss_val = 0.0
    running_accuracy_val = 0.0
    
    # Using itertools for fixed length iteration over non subscriptable DataLoader
    for i, data in enumerate(itertools.islice(train_loader,  STEPS_PER_EPOCH)):
        (context, target_in), target_out = data
        context, target_in, target_out = context.to(device), target_in.to(device), target_out.to(device)
        
        optimizer.zero_grad()
        outputs = translator((context, target_in))
        loss = criterion(target_out, outputs)
        accuracy = acc(target_out, outputs)
        loss.backward()
        optimizer.step()

        running_loss_train += loss.item()
        running_accuracy_train = accuracy 
        
    
    # Validation
    translator.eval()
    with torch.no_grad():
        for i, data in enumerate(itertools.islice(val_loader,  VALIDATION_STEPS)):
            (context, target_in), target_out = data
            context, target_in, target_out = context.to(device), target_in.to(device), target_out.to(device)

            outputs = translator((context, target_in))
            loss = criterion(target_out, outputs)
            accuracy = acc(target_out, outputs)

            running_loss_val += loss.item()
            running_accuracy_val = accuracy 
            
            
    # Print the data
    print(f"\n[epoch: {epoch+1}/{NUM_EPOCHS}] masked_loss: {(running_loss_train / num_batches_train):.4f}, masked_acc: {running_accuracy_train:.4f}, val_masked_loss: {(running_loss_val / num_batches_val):.4f}, val_masked_acc: {running_accuracy_val:.4f}\n")
    
    # Update the best loss if it's better than the previous one
    if running_loss_train < best_loss:
        best_loss = running_loss_train
        patience = 3

    else:
        # Losing patience
        patience -= 1

        if patience == 0:
            print("Early stopping was triggered")

'NUM_EPOCHS = 20\nSTEPS_PER_EPOCH = 500\nVALIDATION_STEPS = 50\npatience = 3\nbest_loss = float(\'inf\')\nnum_batches_train = len(train_loader)\nnum_batches_val = len(val_loader)\n\n\nfor epoch in range(NUM_EPOCHS):\n    translator.train()\n    \n    # Mini batch loss\n    running_loss_train = 0.0\n    running_accuracy_train = 0.0\n    running_loss_val = 0.0\n    running_accuracy_val = 0.0\n    \n    # Using itertools for fixed length iteration over non subscriptable DataLoader\n    for i, data in enumerate(itertools.islice(train_loader,  STEPS_PER_EPOCH)):\n        (context, target_in), target_out = data\n        context, target_in, target_out = context.to(device), target_in.to(device), target_out.to(device)\n        \n        optimizer.zero_grad()\n        outputs = translator((context, target_in))\n        loss = criterion(target_out, outputs)\n        accuracy = acc(target_out, outputs)\n        loss.backward()\n        optimizer.step()\n\n        running_loss_train += loss.item()\

# Using the model for inference

In [24]:
def generate_next_token(context, decoder, next_token, state, done, temperature=0.0):
    logits, state = decoder(context, next_token, state, return_state=True)
    logits = logits[:, -1, :]

    if temperature == 0.0:
        next_token = torch.argmax(logits, dim=-1)

    else:
        logits /= temperature
        distribution = torch.distributions.categorical.Categorical(logits=logits)
        next_token = distribution.sample()

    logits = torch.squeeze(logits)

    next_token = torch.squeeze(next_token)

    logit = logits[next_token].detach().numpy()

    next_token = torch.reshape(next_token, shape=(1,1))
    if next_token == eos_id:
        done = True

    return next_token, logit, state, done

In [25]:
eng_sentence = "I love languages"

context = torch.tensor(encode_sample(eng_sentence))
context = torch.unsqueeze(context, dim=0)
context = encoder(context)

next_token = torch.full((1,1), sos_id)

state = [torch.rand((1, 1, UNITS)), torch.rand((1, 1, UNITS))]
done = False

next_token, logit, state, done = generate_next_token(context, decoder, next_token, state, done, temperature=0.5)
print(f"Next token: {next_token}\nLogit: {logit:.4f}\nDone? {done}")
next_token = next_token.tolist()

Next token: tensor([[8541]])
Logit: -18.7596
Done? False


# Translate

In [26]:
def translate(model, text, max_length=50, temperature=0.0):
    model.eval()
    with torch.no_grad():
        tokens, logits = [], []

        pre_text = text
        text = torch.tensor(encode_sample(pre_text))[None, :]
        context = model.encoder(text)

        next_token = torch.full((1,1), sos_id)

        state = [torch.zeros((1, 1, UNITS)), torch.zeros((1, 1, UNITS))]

        done = False
        for iteration in range(max_length):
            try:
                next_token, logit, state, done = generate_next_token(
                    context=context,
                    decoder=model.decoder,
                    next_token=next_token,
                    state=state,
                    done=done,
                    temperature=temperature
                )
            except:
                raise Exception("Problem generating the next token")

            if done:
                break

            tokens.append(next_token)

            logits.append(logit)

        tokens = torch.cat(tokens, dim=-1).tolist()
    
    translation = ids_to_text(tokens, tokenizer_por)

    return translation, logits[-1], tokens

In [65]:
# Running this cell multiple times should return the same output since temp is 0

temp = 0.0
original_sentence = "I love languages"

translation, logit, tokens = translate(translator.to("cpu"), original_sentence, temperature=temp)

print(f"Temperature: {temp}\n\nOriginal sentence: {original_sentence}\nTranslation: {translation}\nTranslation tokens:{tokens}\nLogit: {logit:.3f}")

Temperature: 0.0

Original sentence: I love languages
Translation: ['eu adoro idiomas com as linguas']
Translation tokens:[[9, 537, 888, 20, 38, 1041]]
Logit: -0.133


In [77]:
# Running this cell multiple times should return different outputs since temp is not 0
# You can try different temperatures

temp = 0.7
original_sentence = "I love languages"

translation, logit, tokens = translate(translator.to("cpu"), original_sentence, temperature=temp)

print(f"Temperature: {temp}\n\nOriginal sentence: {original_sentence}\nTranslation: {translation}\nTranslation tokens:{tokens}\nLogit: {logit:.3f}")

Temperature: 0.7

Original sentence: I love languages
Translation: ['eu adoro linguas']
Translation tokens:[[9, 537, 1041]]
Logit: -1.651


# Minimum Bayes-Risk Decoding

In [78]:
def generate_samples(model, text, n_samples=4, temperature=0.6):
    samples, log_probs = [], []
    
    for _ in range(n_samples):
        _, log_prob, sample = translate(model, text, temperature=temperature)
        
        samples.append(sample)
        
        log_probs.append(log_prob)
        
    return samples, log_probs

In [79]:
samples, log_probs = generate_samples(translator, 'I love languages')

for s, l in zip(samples, log_probs):
    print(f"Translated tensor: {s} has logit: {l:.3f}")

Translated tensor: [[9, 537, 888, 1, 1759, 10120, 9, 9, 7007, 9, 3329, 537, 9, 9696, 9, 578, 9, 537, 1241, 9, 578, 9, 523, 9, 523, 9, 3680, 9, 523, 9, 7007, 9, 9, 523, 9, 523, 9, 537, 9, 523, 9, 9, 523, 9, 1410, 9, 578, 9, 1412, 9]] has logit: -0.665
Translated tensor: [[9, 537, 888, 543, 1041, 3624, 9, 9, 578, 9, 523, 9, 523, 9, 578, 9, 578, 9, 578, 9, 537, 9, 537, 9, 523, 9, 523, 9, 5584, 523, 9, 537, 9, 10490, 523, 9, 807, 9, 523, 9, 578, 9, 523, 9, 523, 9, 578, 9, 523, 9]] has logit: -1.263
Translated tensor: [[9, 537, 888, 166]] has logit: -5.841
Translated tensor: [[9, 537, 888, 4651, 12, 1041]] has logit: -4.314


# Comparing overlaps

In [80]:
def jaccard_similarity(candidate, reference):
    
    if (isinstance(candidate, list) and all(isinstance(i, list) for i in candidate)) and \
       (isinstance(reference, list) and all(isinstance(i, list) for i in reference)):
        candidate_set = set(candidate[0])
        reference_set = set(reference[0])

    else:
        candidate_set = set(candidate)
        reference_set = set(reference)    
    
    common_tokens = candidate_set.intersection(reference_set)
    
    all_tokens = candidate_set.union(reference_set)
    
    overlap = len(common_tokens) / len(all_tokens)
    
    return overlap

In [81]:
l1 = [1,2,3]
l2 = [1,2,3,4]

js = jaccard_similarity(l1, l2)

print(f"jaccard similarity between lists: {l1} and {l2} is {js:.3f}")

jaccard similarity between lists: [1, 2, 3] and [1, 2, 3, 4] is 0.750


# Rouge1 similarity

In [82]:
def rouge1_similarity(candidate, reference):
    candidate_word_counts = Counter(candidate)
    reference_word_counts = Counter(reference)    
    
    overlap = 0
    
    for token in candidate_word_counts.keys():
        token_count_candidate = candidate_word_counts[token]
        token_count_reference = reference_word_counts[token]        
        
        overlap += min(token_count_candidate, token_count_reference)
        
    precision = overlap / len(candidate)
    
    recall = overlap / len(reference)
    
    if precision + recall != 0:
        f1_score = 2 * (precision * recall) / (precision + recall)
        return f1_score
    
    return 0

In [83]:
l1 = [0, 1]
l2 = [5, 5, 7, 0, 232]

r1s = rouge1_similarity(l1, l2)

print(f"rouge 1 similarity between lists: {l1} and {l2} is {r1s:.3f}")

rouge 1 similarity between lists: [0, 1] and [5, 5, 7, 0, 232] is 0.286


In [84]:
l1 = [1, 2, 3]
l2 = [1, 2, 3, 4]

r1s = rouge1_similarity(l1, l2)

print(f"rouge 1 similarity between lists: {l1} and {l2} is {r1s:.3f}")

rouge 1 similarity between lists: [1, 2, 3] and [1, 2, 3, 4] is 0.857


# Computing the overall score

# Average overlap

In [85]:
def average_overlap(samples, similarity_fn):
    
    scores = {}
    
    for index_candidate, candidate in enumerate(samples):
        overlap = 0
        
        for index_sample, sample in enumerate(samples):
            
            if index_candidate == index_sample:
                continue
                
            overlap += similarity_fn(candidate, sample)
            
        score = overlap / (len(samples) - 1)
        
        score = round(score, 3)
        
        scores[index_candidate] = score
        
    return scores

In [86]:
# Test with Jaccard similarity

l1 = [1, 2, 3]
l2 = [1, 2, 4]
l3 = [1, 2, 4, 5]

avg_ovlp = average_overlap([l1, l2, l3], jaccard_similarity)

print(f"average overlap between lists: {l1}, {l2} and {l3} using Jaccard similarity is:\n\n{avg_ovlp}")

average overlap between lists: [1, 2, 3], [1, 2, 4] and [1, 2, 4, 5] using Jaccard similarity is:

{0: 0.45, 1: 0.625, 2: 0.575}


In [87]:
# Test with Rouge1 similarity

l1 = [1, 2, 3]
l2 = [1, 4]
l3 = [1, 2, 4, 5]
l4 = [5,6]

avg_ovlp = average_overlap([l1, l2, l3, l4], rouge1_similarity)

print(f"average overlap between lists: {l1}, {l2}, {l3} and {l4} using Rouge1 similarity is:\n\n{avg_ovlp}")

average overlap between lists: [1, 2, 3], [1, 4], [1, 2, 4, 5] and [5, 6] using Rouge1 similarity is:

{0: 0.324, 1: 0.356, 2: 0.524, 3: 0.111}


In [88]:
def weighted_avg_overlap(samples, log_probs, similarity_fn):
    scores = {}
    
    for index_candidate, candidate in enumerate(samples):
        overlap, weighted_sum = 0.0, 0.0
        
        for index_sample, (sample, logprob) in enumerate(zip(samples, log_probs)):
            if index_candidate == index_sample:
                continue
                
            sample_prob = float(np.exp(logprob))
            weighted_sum += sample_prob
            
            sample_overlap = similarity_fn(candidate, sample)
            overlap += sample_overlap * sample_prob
            
        score = overlap / weighted_sum
        score = round(score, 3)
        
        scores[index_candidate] = score
        
    return scores

In [89]:
l1 = [1, 2, 3]
l2 = [1, 2, 4]
l3 = [1, 2, 4, 5]
log_probs = [0.4, 0.2, 0.5]

w_avg_ovlp = weighted_avg_overlap([l1, l2, l3], log_probs, jaccard_similarity)

print(f"weighted average overlap using Jaccard similarity is:\n\n{w_avg_ovlp}")

weighted average overlap using Jaccard similarity is:

{0: 0.443, 1: 0.631, 2: 0.558}


In [90]:
def mbr_decode(model, text, n_samples=5, temperature=0.6, similarity_fn=jaccard_similarity):
    samples, log_probs = generate_samples(model, text, n_samples=n_samples, temperature=temperature)
    
    scores = weighted_avg_overlap(samples, log_probs, similarity_fn)
    
    decoded_translations = [ids_to_text(sample,tokenizer_por) for sample in samples]
    
    max_score_key = max(scores, key=lambda k: scores[k])
    
    translation = decoded_translations[max_score_key]
    
    return translation, decoded_translations

In [94]:
english_sentence = "I love languages"

translation, candidates = mbr_decode(translator, english_sentence, n_samples=10, temperature=0.6)

print("Translation candidates:")
for c in candidates:
    print(c)

print(f"\nSelected translation: {translation}")

Translation candidates:
['eu adoro idiomas']
['eu adoro idiomas para os idiomas eu eu amo eu odeio eu odeio eu adoro eu amo eu amo eu adoro eu amo eu beisebol eu amo eu odeio eu amo eu odeio eu adorei eu amo eu amo eu amo eu eu amo amo amo eu amo eu']
['eu amo idiomas boa deveras meu dicas eu amo eu amo eu circo eu amo eu amo eu encontrei eu eu gramado eu amo eu amo eu adoro eu virem adoram eu amo eu naturalmente eu amo eu adoro eu amo eu eu amo eu dirija eu amo eu amo']
['eu adoro idiomas muito palavras obrigado eu eu adoro eu amo eu amo eu vinda adoro eu amo eu amo eu amo eu amo amo eu adoro eu adorei eu adoro eu amo eu amo eu amo eu adoro eu amo eu amo eu amo eu amo eu b eu']
['eu amo linguas falas']
['eu amo linguas perguntar']
['eu amo linguas segura']
['eu amo linguas']
['eu adoro linguas falamos']
['eu adoro linguas']

Selected translation: ['eu adoro linguas']
